In [1]:
from datetime import datetime, date
import os
import re
import pickle

import pandas as pd
from sklearn.pipeline import Pipeline
import mlflow

import utils.postprocessing_lib_rodrixx as post
import utils.preprocessing_lib_rodrixx as prep

import warnings
warnings.filterwarnings('ignore')

In [2]:
dfs_path = os.path.join(os.getcwd(), 'dataframes')
df = pd.read_pickle(os.path.join(dfs_path, 'stats_1982_to_2022.pkl'))

In [3]:
path_rf = 'runs:/ca376189801e48cd905ced1f13f73600/model'
path_xgb = 'runs:/f217bfcb4a094f648e661a3400747eaa/model'
path_ens = 'runs:/ab01e06711554c1d8e7695357f2d9714/model'

In [4]:
model_rf = mlflow.pyfunc.load_model(path_rf)

In [5]:
cols_tot = [col for col in df.columns if '_tot' in col]
cols_to_drop = ['Rk', 'G', 'GS', 'GT', 'Tm', 'Votes', 'MaxVotes', 'Share']
cols_to_drop += cols_tot
col_to_ohe = 'Pos'

In [8]:
pipe_prep = Pipeline(steps = [
    ('DropPlayersMultiTeams', prep.DropPlayersMultiTeams()),
    ('SetIndex', prep.SetIndex()),
    ('OHE', prep.OHE(col_to_ohe)),
    ('DropColumns', prep.DropColumns(cols_to_drop)),
    ('DropPlayers', prep.DropPlayers()),
])

In [9]:
pre_df = pipe_prep.fit_transform(df)